# Chapter 16: Generics in Depth

---

## 16.1 Generic Interfaces with Multiple Type Parameters

Generic interfaces can accept multiple type parameters, enabling sophisticated type relationships and reusable abstractions.

### 16.1.1 Defining Multiple Type Parameters

**Multi-Parameter Generic Interfaces:**

```typescript
// Interface with two type parameters
interface Pair<K, V> {
  key: K;
  value: V;
  toString(): string;
  equals(other: Pair<K, V>): boolean;
}

// Usage with different combinations
const stringNumberPair: Pair<string, number> = {
  key: "age",
  value: 30,
  toString() {
    return `${this.key}: ${this.value}`;
  },
  equals(other) {
    return this.key === other.key && this.value === other.value;
  }
};

const numberBooleanPair: Pair<number, boolean> = {
  key: 1,
  value: true,
  toString() {
    return `${this.key}: ${this.value}`;
  },
  equals(other) {
    return this.key === other.key && this.value === other.value;
  }
};

// Interface with three type parameters
interface Triplet<A, B, C> {
  first: A;
  second: B;
  third: C;
  toArray(): [A, B, C];
}

const triplet: Triplet<string, number, boolean> = {
  first: "hello",
  second: 42,
  third: true,
  toArray() {
    return [this.first, this.second, this.third];
  }
};

// Generic interface for conversion
interface Converter<Input, Output, Options = {}> {
  convert(input: Input, options?: Options): Output;
  reverse?(output: Output): Input;
}

// String to number converter
const stringToNumber: Converter<string, number, { radix?: number }> = {
  convert(input, options) {
    return parseInt(input, options?.radix ?? 10);
  },
  reverse(output) {
    return output.toString();
  }
};

// Object to JSON converter
const toJSON: Converter<object, string, { spaces?: number }> = {
  convert(input, options) {
    return JSON.stringify(input, null, options?.spaces);
  }
};
```

### 16.1.2 Implementing Multi-Parameter Interfaces

**Class Implementation:**

```typescript
// Generic interface for key-value storage
interface Storage<K, V> {
  set(key: K, value: V): void;
  get(key: K): V | undefined;
  has(key: K): boolean;
  delete(key: K): boolean;
  clear(): void;
  keys(): K[];
  values(): V[];
  entries(): [K, V][];
}

// Implementation with specific constraints
class TypedMap<K extends string | number, V> implements Storage<K, V> {
  private data = new Map<K, V>();
  
  set(key: K, value: V): void {
    this.data.set(key, value);
  }
  
  get(key: K): V | undefined {
    return this.data.get(key);
  }
  
  has(key: K): boolean {
    return this.data.has(key);
  }
  
  delete(key: K): boolean {
    return this.data.delete(key);
  }
  
  clear(): void {
    this.data.clear();
  }
  
  keys(): K[] {
    return Array.from(this.data.keys());
  }
  
  values(): V[] {
    return Array.from(this.data.values());
  }
  
  entries(): [K, V][] {
    return Array.from(this.data.entries());
  }
}

// Usage
const userMap = new TypedMap<number, { name: string; email: string }>();
userMap.set(1, { name: "John", email: "john@example.com" });
const user = userMap.get(1); // Type: { name: string; email: string } | undefined

const configMap = new TypedMap<string, boolean>();
configMap.set("darkMode", true);
const darkMode = configMap.get("darkMode"); // Type: boolean | undefined

// Complex multi-parameter interface
interface Repository<T, ID, Query = Partial<T>, CreateDTO = Omit<T, "id">> {
  findById(id: ID): Promise<T | null>;
  findAll(query?: Query): Promise<T[]>;
  create(data: CreateDTO): Promise<T>;
  update(id: ID, data: Partial<T>): Promise<T>;
  delete(id: ID): Promise<boolean>;
}

// Implementation
class UserRepository implements Repository<
  { id: number; name: string; email: string },
  number,
  { name?: string; email?: string },
  { name: string; email: string }
> {
  private users: Map<number, { id: number; name: string; email: string }> = new Map();
  private nextId = 1;
  
  async findById(id: number): Promise<{ id: number; name: string; email: string } | null> {
    return this.users.get(id) ?? null;
  }
  
  async findAll(query?: { name?: string; email?: string }): Promise<{ id: number; name: string; email: string }[]> {
    let results = Array.from(this.users.values());
    if (query?.name) {
      results = results.filter(u => u.name.includes(query.name!));
    }
    if (query?.email) {
      results = results.filter(u => u.email.includes(query.email!));
    }
    return results;
  }
  
  async create(data: { name: string; email: string }): Promise<{ id: number; name: string; email: string }> {
    const user = { ...data, id: this.nextId++ };
    this.users.set(user.id, user);
    return user;
  }
  
  async update(id: number, data: Partial<{ id: number; name: string; email: string }>): Promise<{ id: number; name: string; email: string }> {
    const existing = this.users.get(id);
    if (!existing) throw new Error("Not found");
    const updated = { ...existing, ...data };
    this.users.set(id, updated);
    return updated;
  }
  
  async delete(id: number): Promise<boolean> {
    return this.users.delete(id);
  }
}
```

---

## 16.2 Generic Classes with Inheritance

Generic classes can extend other generic classes, creating powerful inheritance hierarchies that maintain type safety across the inheritance chain.

### 16.2.1 Extending Generic Classes

**Generic Inheritance Patterns:**

```typescript
// Base generic class
class Container<T> {
  protected items: T[] = [];
  
  constructor(initialItems: T[] = []) {
    this.items = [...initialItems];
  }
  
  add(item: T): void {
    this.items.push(item);
  }
  
  getAll(): T[] {
    return [...this.items];
  }
  
  isEmpty(): boolean {
    return this.items.length === 0;
  }
}

// Extended generic class with additional constraint
class SortableContainer<T extends number | string> extends Container<T> {
  sort(): T[] {
    return [...this.items].sort((a, b) => {
      if (a < b) return -1;
      if (a > b) return 1;
      return 0;
    });
  }
  
  getMin(): T | undefined {
    if (this.isEmpty()) return undefined;
    return this.sort()[0];
  }
  
  getMax(): T | undefined {
    if (this.isEmpty()) return undefined;
    return this.sort()[this.items.length - 1];
  }
}

// Usage
const numbers = new SortableContainer<number>([3, 1, 4, 1, 5]);
numbers.add(2);
console.log(numbers.sort()); // [1, 1, 2, 3, 4, 5]
console.log(numbers.getMin()); // 1
console.log(numbers.getMax()); // 5

const strings = new SortableContainer<string>(["banana", "apple", "cherry"]);
console.log(strings.sort()); // ["apple", "banana", "cherry"]

// const mixed = new SortableContainer<boolean>(); // ❌ Error: boolean doesn't extend number | string

// Generic class extending with additional type parameter
class KeyedContainer<K, V> extends Container<V> {
  private keyMap = new Map<K, V>();
  
  addWithKey(key: K, value: V): void {
    this.add(value);
    this.keyMap.set(key, value);
  }
  
  getByKey(key: K): V | undefined {
    return this.keyMap.get(key);
  }
  
  hasKey(key: K): boolean {
    return this.keyMap.has(key);
  }
}

const keyed = new KeyedContainer<string, number>();
keyed.addWithKey("one", 1);
keyed.addWithKey("two", 2);
console.log(keyed.getByKey("one")); // 1
```

### 16.2.2 Generic Constraints in Inheritance

**Constrained Generic Inheritance:**

```typescript
// Interface for identifiable items
interface Identifiable {
  id: string | number;
}

// Base class with constraint
class RepositoryBase<T extends Identifiable> {
  protected items: Map<string | number, T> = new Map();
  
  findById(id: string | number): T | undefined {
    return this.items.get(id);
  }
  
  save(item: T): void {
    this.items.set(item.id, item);
  }
  
  delete(id: string | number): boolean {
    return this.items.delete(id);
  }
  
  getAll(): T[] {
    return Array.from(this.items.values());
  }
}

// Extended class with stricter constraint
interface Auditable extends Identifiable {
  createdAt: Date;
  updatedAt: Date;
}

class AuditableRepository<T extends Auditable> extends RepositoryBase<T> {
  save(item: T): void {
    const now = new Date();
    if (!item.createdAt) {
      item.createdAt = now;
    }
    item.updatedAt = now;
    super.save(item);
  }
  
  getRecentlyUpdated(since: Date): T[] {
    return this.getAll().filter(item => item.updatedAt >= since);
  }
}

// Even more specific extension
interface Versioned extends Auditable {
  version: number;
}

class VersionedRepository<T extends Versioned> extends AuditableRepository<T> {
  save(item: T): void {
    if (!item.version) {
      item.version = 1;
    }
    super.save(item);
  }
  
  incrementVersion(id: string | number): T | undefined {
    const item = this.findById(id);
    if (item) {
      item.version++;
      item.updatedAt = new Date();
      this.save(item);
    }
    return item;
  }
}

// Usage
interface Document extends Versioned {
  id: string;
  title: string;
  content: string;
}

const docRepo = new VersionedRepository<Document>();
docRepo.save({
  id: "doc-1",
  title: "Hello",
  content: "World",
  createdAt: new Date(),
  updatedAt: new Date(),
  version: 1
});

docRepo.incrementVersion("doc-1");
```

### 16.2.3 Overriding Generic Methods

**Method Override Patterns:**

```typescript
// Base generic class
class DataProcessor<TInput, TOutput> {
  protected transform: (input: TInput) => TOutput;
  
  constructor(transform: (input: TInput) => TOutput) {
    this.transform = transform;
  }
  
  process(input: TInput): TOutput {
    return this.transform(input);
  }
  
  processBatch(inputs: TInput[]): TOutput[] {
    return inputs.map(input => this.process(input));
  }
}

// Specialized processor with narrowed types
class StringProcessor extends DataProcessor<string, number> {
  constructor() {
    super(s => s.length);
  }
  
  // Override with specific implementation
  process(input: string): number {
    console.log(`Processing: ${input}`);
    return super.process(input);
  }
  
  // Additional specialized method
  processLines(text: string): number[] {
    return text.split('\n').map(line => this.process(line));
  }
}

// Generic override with different constraints
class ValidatedProcessor<T extends { validate(): boolean }, R> 
  extends DataProcessor<T, R> {
  
  process(input: T): R {
    if (!input.validate()) {
      throw new Error("Invalid input");
    }
    return super.process(input);
  }
}

interface ValidatableString {
  value: string;
  validate(): boolean;
}

class ValidatedStringProcessor extends ValidatedProcessor<ValidatableString, number> {
  constructor() {
    super(s => s.value.length);
  }
}

const processor = new ValidatedStringProcessor();
const result = processor.process({ value: "hello", validate: () => true });
```

---

## 16.3 Generic Type Aliases

Type aliases can be generic, providing flexible ways to create reusable type transformations.

### 16.3.1 Defining Generic Type Aliases

**Generic Alias Patterns:**

```typescript
// Generic container type
type Container<T> = {
  value: T;
  get(): T;
  set(newValue: T): void;
};

const stringContainer: Container<string> = {
  value: "hello",
  get() { return this.value; },
  set(v) { this.value = v; }
};

// Generic promise type
type AsyncResult<T> = Promise<{ data: T; timestamp: Date }>;

async function fetchData(): AsyncResult<User[]> {
  const response = await fetch("/api/users");
  const data = await response.json();
  return { data, timestamp: new Date() };
}

interface User {
  id: number;
  name: string;
}

// Generic mapped type alias
type Nullable<T> = {
  [K in keyof T]: T[K] | null;
};

type UserWithNulls = Nullable<User>;
// { id: number | null; name: string | null }

// Generic conditional type alias
type NonNullable<T> = T extends null | undefined ? never : T;

type StringOrNull = string | null;
type JustString = NonNullable<StringOrNull>; // string

// Generic union with constraints
type EventPayload<T extends string> = {
  type: T;
  timestamp: number;
  data: unknown;
};

type ClickEvent = EventPayload<"click"> & { data: { x: number; y: number } };
type SubmitEvent = EventPayload<"submit"> & { data: { formData: FormData } };

// Generic function type alias
type Reducer<S, A> = (state: S, action: A) => S;
type Dispatch<A> = (action: A) => void;

const counterReducer: Reducer<number, { type: "increment" | "decrement" }> = 
  (state, action) => {
    switch (action.type) {
      case "increment": return state + 1;
      case "decrement": return state - 1;
    }
  };
```

### 16.3.2 Generic Aliases with Constraints

**Constrained Generic Aliases:**

```typescript
// Alias with constraint
type StringKeyOf<T extends Record<string, any>> = Extract<keyof T, string>;

interface Person {
  name: string;
  age: number;
  0: string; // Numeric key
}

type PersonStringKeys = StringKeyOf<Person>; // "name" | "age"

// Alias ensuring property exists
type EnsureProperty<T, K extends keyof T> = T & Required<Pick<T, K>>;

interface Config {
  apiUrl?: string;
  timeout?: number;
}

const fullConfig: EnsureProperty<Config, "apiUrl"> = {
  apiUrl: "https://api.example.com", // Required
  timeout: 5000 // Optional
};

// Generic alias for API responses
type ApiResponse<T, E = { message: string; code: number }> =
  | { status: "success"; data: T }
  | { status: "error"; error: E };

type UserResponse = ApiResponse<{ users: User[] }>;
type ErrorResponse = ApiResponse<never, { message: string; code: number; details: unknown }>;

// Generic alias for event handlers
type EventHandler<TEvent extends Event> = (event: TEvent) => void;
type ClickHandler = EventHandler<MouseEvent>;
type ChangeHandler = EventHandler<Event & { target: HTMLInputElement }>;

// Generic alias for factory functions
type Factory<T, Args extends any[] = []> = (...args: Args) => T;

const createUser: Factory<User, [string, string]> = (name, email) => ({
  id: Math.random(),
  name,
  email
});
```

---

## 16.4 Generic Constraints with `keyof`

The `keyof` operator combined with generics creates powerful patterns for type-safe property access and manipulation.

### 16.4.1 `keyof` Operator

**Understanding `keyof`:**

```typescript
// keyof produces a union of property keys
interface Person {
  name: string;
  age: number;
  email: string;
}

type PersonKeys = keyof Person; // "name" | "age" | "email"

// Using keyof in generics
function getProperty<T, K extends keyof T>(obj: T, key: K): T[K] {
  return obj[key];
}

const person: Person = {
  name: "John",
  age: 30,
  email: "john@example.com"
};

const name = getProperty(person, "name"); // Type: string
const age = getProperty(person, "age");   // Type: number
// const invalid = getProperty(person, "phone"); // ❌ Error: "phone" not in keyof Person

// keyof with index signatures
interface Dictionary {
  [key: string]: number;
}

type DictKeys = keyof Dictionary; // string | number
// (includes number because dictionary keys are coerced to string)

// keyof with unions
type A = { a: string; b: number };
type B = { b: boolean; c: string };
type AorBKeys = keyof (A | B); // "b" (only common property)

// keyof with intersections
type AandBKeys = keyof (A & B); // "a" | "b" | "c" (all properties)
```

### 16.4.2 Combining Generics with `keyof`

**Type-Safe Property Access:**

```typescript
// Safe property getter
function getPropertySafe<T, K extends keyof T>(
  obj: T,
  key: K,
  defaultValue: T[K]
): T[K] {
  return obj[key] ?? defaultValue;
}

// Safe property setter with validation
function setProperty<T, K extends keyof T>(
  obj: T,
  key: K,
  value: T[K]
): void {
  // Could add validation logic here
  obj[key] = value;
}

// Pick multiple properties
function pick<T, K extends keyof T>(obj: T, keys: K[]): Pick<T, K> {
  const result = {} as Pick<T, K>;
  keys.forEach(key => {
    result[key] = obj[key];
  });
  return result;
}

const person = {
  name: "John",
  age: 30,
  email: "john@example.com",
  city: "NYC"
};

const picked = pick(person, ["name", "email"]);
// Type: { name: string; email: string }

// Omit multiple properties (inverse of pick)
function omit<T, K extends keyof T>(obj: T, keys: K[]): Omit<T, K> {
  const result = { ...obj };
  keys.forEach(key => {
    delete (result as any)[key];
  });
  return result;
}

const omitted = omit(person, ["age", "city"]);
// Type: { name: string; email: string }

// Get keys by value type
type KeysOfType<T, U> = {
  [K in keyof T]: T[K] extends U ? K : never;
}[keyof T];

type StringKeys = KeysOfType<Person, string>; // "name" | "email"
type NumberKeys = KeysOfType<Person, number>; // "age"

// Function to pluck specific value type
function pluck<T, K extends KeysOfType<T, string>>(obj: T, key: K): string {
  return obj[key] as string;
}

const personName = pluck(person, "name"); // Type: string
```

---

## 16.5 Default Type Parameters

Generic type aliases and interfaces can specify default values for type parameters, making them optional when appropriate.

### 16.5.1 Default Type Parameter Syntax

**Setting Defaults:**

```typescript
// Generic type alias with defaults
type ApiResponse<T = unknown, E = Error> =
  | { status: "success"; data: T }
  | { status: "error"; error: E };

// Uses defaults: T = unknown, E = Error
const response1: ApiResponse = {
  status: "success",
  data: { anything: "goes" } // data is unknown
};

// Specify T, use default E
const response2: ApiResponse<User> = {
  status: "success",
  data: { id: 1, name: "John" }
};

// Specify both
const response3: ApiResponse<User, CustomError> = {
  status: "error",
  error: { code: 404, message: "Not found" }
};

interface User {
  id: number;
  name: string;
}

interface CustomError {
  code: number;
  message: string;
}

// Generic interface with defaults
interface Cache<T = string, K = string> {
  get(key: K): T | undefined;
  set(key: K, value: T): void;
  has(key: K): boolean;
  clear(): void;
}

// Uses defaults
const stringCache: Cache = {
  get(key) { return undefined; },
  set(key, value) {},
  has(key) { return false; },
  clear() {}
};

// Specify types
const numberCache: Cache<number, number> = {
  get(key) { return undefined; },
  set(key, value) {},
  has(key) { return false; },
  clear() {}
};
```

### 16.5.2 Default Parameters with Constraints

**Constrained Defaults:**

```typescript
// Default with constraint
interface Container<T extends object = {}> {
  data: T;
  getData(): T;
}

const defaultContainer: Container = {
  data: {},
  getData() { return this.data; }
};

const specificContainer: Container<{ name: string }> = {
  data: { name: "John" },
  getData() { return this.data; }
};

// Multiple defaults with constraints
type EventHandler<T extends Event = Event, R = void> = (event: T) => R;

const defaultHandler: EventHandler = (e) => {
  console.log(e.type); // e is Event
};

const clickHandler: EventHandler<MouseEvent, boolean> = (e) => {
  console.log(e.clientX); // e is MouseEvent
  return true;
};

// Default based on another parameter
type MapConfig<K = string, V = K extends string ? string : unknown> = {
  keyType: K;
  valueType: V;
  entries: Map<K, V>;
};

const stringMap: MapConfig = {
  keyType: "string",
  valueType: "string",
  entries: new Map()
};

const numberMap: MapConfig<number, boolean> = {
  keyType: 1,
  valueType: true,
  entries: new Map()
};
```

---

## 16.6 Conditional Types with Generics

Conditional types allow you to create types that depend on a condition, enabling powerful type transformations.

### 16.6.1 Conditional Type Basics

**Conditional Type Syntax:**

```typescript
// Basic conditional type
type IsString<T> = T extends string ? true : false;

type A = IsString<string>;  // true
type B = IsString<number>;  // false
type C = IsString<"hello">; // true (literal extends string)

// Conditional type with generics
type NonNullable<T> = T extends null | undefined ? never : T;

type MaybeString = string | null | undefined;
type DefinitelyString = NonNullable<MaybeString>; // string

// Extract from union
type Extract<T, U> = T extends U ? T : never;

type StringOrNumber = string | number | boolean;
type JustStrings = Extract<StringOrNumber, string>; // string

// Exclude from union
type Exclude<T, U> = T extends U ? never : T;

type WithoutStrings = Exclude<StringOrNumber, string>; // number | boolean

// Conditional with objects
type HasProperty<T, K extends PropertyKey> = K extends keyof T ? true : false;

interface Person {
  name: string;
  age: number;
}

type HasName = HasProperty<Person, "name">; // true
type HasPhone = HasProperty<Person, "phone">; // false
```

### 16.6.2 Inferring with Conditional Types

**The `infer` Keyword:**

```typescript
// Extract return type
type ReturnType<T> = T extends (...args: any[]) => infer R ? R : never;

function getUser() {
  return { id: 1, name: "John" };
}

type UserReturn = ReturnType<typeof getUser>; // { id: number; name: string }

// Extract parameter types
type Parameters<T> = T extends (...args: infer P) => any ? P : never;

function updateUser(id: number, data: { name: string }) {}

type UpdateParams = Parameters<typeof updateUser>; // [number, { name: string }]

// Extract array element type
type ElementType<T> = T extends (infer E)[] ? E : never;

type Numbers = number[];
type Num = ElementType<Numbers>; // number

// Extract promise value
type PromiseType<T> = T extends Promise<infer V> ? V : never;

async function fetchData(): Promise<{ users: User[] }> {
  return { users: [] };
}

type FetchResult = PromiseType<ReturnType<typeof fetchData>>; // { users: User[] }

// Extract function type from class
type ConstructorParameters<T> = T extends new (...args: infer P) => any ? P : never;
type InstanceType<T> = T extends new (...args: any[]) => infer R ? R : never;

class MyClass {
  constructor(public name: string, public age: number) {}
}

type MyClassParams = ConstructorParameters<typeof MyClass>; // [string, number]
type MyClassInstance = InstanceType<typeof MyClass>; // MyClass
```

---

## 16.7 Variance Annotations

TypeScript 4.7+ introduced explicit variance annotations for type parameters, allowing you to control how generic types relate to each other in terms of substitutability.

### 16.7.1 Covariance

**Covariant (`out`) Annotation:**

```typescript
// Covariant: if Dog extends Animal, then Container<Dog> extends Container<Animal>
// (produces "out" values - you can only read T)

interface Animal {
  name: string;
}

interface Dog extends Animal {
  bark(): void;
}

// Mark T as covariant (out)
interface Producer<out T> {
  produce(): T;
  // Cannot have methods that accept T as parameter
  // consume(item: T): void; // ❌ Error: T is covariant
}

// Covariant assignment
const dogProducer: Producer<Dog> = {
  produce: () => ({ name: "Rex", bark: () => {} })
};

const animalProducer: Producer<Animal> = dogProducer; // ✅ OK - covariance
// Producer<Dog> is assignable to Producer<Animal>

// Real-world example: Read-only data source
interface DataSource<out T> {
  read(): T;
  onData(callback: (data: T) => void): void;
}

const stringSource: DataSource<string> = {
  read: () => "data",
  onData: (cb) => cb("data")
};

const unknownSource: DataSource<unknown> = stringSource; // OK
```

### 16.7.2 Contravariance

**Contravariant (`in`) Annotation:**

```typescript
// Contravariant: if Dog extends Animal, then Handler<Animal> extends Handler<Dog>
// (accepts "in" values - you can only write/consume T)

// Mark T as contravariant (in)
interface Consumer<in T> {
  consume(item: T): void;
  // Cannot have methods that return T
  // produce(): T; // ❌ Error: T is contravariant
}

const animalConsumer: Consumer<Animal> = {
  consume: (animal) => console.log(animal.name)
};

const dogConsumer: Consumer<Dog> = animalConsumer; // ✅ OK - contravariance
// Consumer<Animal> is assignable to Consumer<Dog>

// Real-world example: Event handlers
interface EventHandler<in T> {
  handle(event: T): void;
}

const genericHandler: EventHandler<Event> = {
  handle: (e) => console.log(e.type)
};

const specificHandler: EventHandler<MouseEvent> = genericHandler; // OK
```

### 16.7.3 Invariance

**Invariant (no annotation):**

```typescript
// Invariant: Container<Dog> is NOT assignable to Container<Animal>
// and Container<Animal> is NOT assignable to Container<Dog>
// (can both read and write T)

interface MutableContainer<T> {
  get(): T;
  set(value: T): void;
}

const dogContainer: MutableContainer<Dog> = {
  get: () => ({ name: "Rex", bark: () => {} }),
  set: (dog) => {}
};

// Neither of these assignments work with invariance
// const animalContainer: MutableContainer<Animal> = dogContainer; // ❌ Error
// const specificDogContainer: MutableContainer<Dog> = animalContainer; // ❌ Error

// Invariance is the safest default but most restrictive
```

### 16.7.4 Explicit Variance Annotations

**Variance Syntax:**

```typescript
// Covariant (out) - can only produce T
interface Source<out T> {
  next(): T;
}

// Contravariant (in) - can only consume T
interface Sink<in T> {
  write(value: T): void;
}

// Invariant (no annotation or in out) - can do both
interface Storage<T> {
  get(): T;
  set(value: T): void;
}

// Bivariant (rare, unsafe) - TypeScript allows for function parameters
// Not explicitly annotatable, but function parameters are bivariant with strictFunctionTypes off

// Practical example with variance
interface Comparable<in T> {
  compareTo(other: T): number;
}

// Comparable is contravariant in T because it accepts T as parameter
const animalComparable: Comparable<Animal> = {
  compareTo: (other) => other.name.localeCompare("test")
};

const dogComparable: Comparable<Dog> = animalComparable; // OK due to contravariance
```

---

## 16.8 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored advanced generic patterns:

**Key Takeaways:**

1. **Multi-Parameter Generics**: Interfaces and classes can accept multiple type parameters for complex relationships.

2. **Generic Inheritance**: Classes can extend generic classes, with the ability to add constraints or additional type parameters.

3. **Generic Type Aliases**: Type aliases can be generic, providing flexible type transformations and utility types.

4. **`keyof` Constraints**: Combining generics with `keyof` enables type-safe property access and manipulation.

5. **Default Type Parameters**: Type parameters can have defaults, making them optional when sensible.

6. **Conditional Types**: `T extends U ? X : Y` enables type-level logic and type extraction.

7. **Variance Annotations**: `in` (contravariant), `out` (covariant) control substitutability of generic types.

### Practical Exercises

**Exercise 1: Multi-Parameter Generics**

Build a type-safe event system:

```typescript
// 1. Create a generic interface EventEmitter<Events extends Record<string, any>> with:
//    - on<K extends keyof Events>(event: K, listener: (data: Events[K]) => void): void
//    - off<K extends keyof Events>(event: K, listener: (data: Events[K]) => void): void
//    - emit<K extends keyof Events>(event: K, data: Events[K]): void

// 2. Define an Events type:
//    type AppEvents = {
//      userLogin: { userId: string; timestamp: Date };
//      userLogout: { userId: string };
//      dataUpdate: { table: string; rows: number };
//    }

// 3. Create an instance and demonstrate type-safe event handling

// 4. Add a once<K>(event, listener) method that removes listener after first call
```

**Exercise 2: Generic Inheritance**

Create a data layer:

```typescript
// 1. Create base interface Entity { id: string | number; }

// 2. Create base class Repository<T extends Entity> with CRUD operations

// 3. Create AuditableEntity extends Entity with createdAt and updatedAt

// 4. Create AuditableRepository<T extends AuditableEntity> that:
//    - Extends Repository<T>
//    - Automatically sets timestamps on save
//    - Adds findByDateRange method

// 5. Create VersionedEntity extends AuditableEntity with version: number

// 6. Create VersionedRepository with optimistic concurrency control

// 7. Demonstrate with concrete entity types
```

**Exercise 3: keyof and Generics**

Implement utility functions:

```typescript
// 1. Implement groupBy<T, K extends keyof T>(array: T[], key: K): Map<T[K], T[]>

// 2. Implement indexBy<T, K extends keyof T>(array: T[], key: K): Map<T[K], T>

// 3. Implement pluck<T, K extends keyof T>(array: T[], key: K): T[K][]

// 4. Implement setNested<T, K extends keyof T>(obj: T, key: K, value: T[K]): T
//    (returns new object, doesn't mutate)

// 5. Implement a type-safe pick that requires at least one key:
//    pick<T, K extends keyof T>(obj: T, key: K, ...keys: K[]): Pick<T, K>
```

**Exercise 4: Conditional Types**

Build type utilities:

```typescript
// 1. Implement DeepPartial<T> that makes all properties optional recursively

// 2. Implement DeepRequired<T> that makes all properties required recursively

// 3. Implement DeepReadonly<T> that makes all properties readonly recursively

// 4. Implement Flatten<T> that flattens array types: T[][] => T[]

// 5. Implement UnwrapPromise<T> that extracts value from Promise

// 6. Implement Parameters<T> and ReturnType<T> from scratch using infer

// 7. Test with complex nested types
```

**Exercise 5: Variance and Advanced Patterns**

Create a type-safe dependency injection container:

```typescript
// 1. Create interfaces:
//    - Provider<T> with get(): T (covariant out)
//    - Consumer<T> with inject(value: T): void (contravariant in)

// 2. Create a Container class that:
//    - Registers providers by token
//    - Resolves dependencies
//    - Maintains singleton scope

// 3. Create decorator-like functions:
//    - singleton<T>(factory: () => T): Provider<T>
//    - transient<T>(factory: () => T): Provider<T>

// 4. Demonstrate variance:
//    - Show that Provider<Dog> can be used as Provider<Animal>
//    - Show that Consumer<Animal> can be used as Consumer<Dog>

// 5. Add lifecycle hooks with proper typing
```

### Additional Resources

- **TypeScript Handbook - Generics**: https://www.typescriptlang.org/docs/handbook/2/generics.html
- **Conditional Types**: https://www.typescriptlang.org/docs/handbook/2/conditional-types.html
- **Type Inference**: https://www.typescriptlang.org/docs/handbook/type-inference.html
- **Variance Annotations**: https://www.typescriptlang.org/docs/handbook/release-notes/typescript-4-7.html#optional-variance-annotations-for-type-parameters

---

## Coming Up Next: Chapter 17 - Generic Patterns and Best Practices

In the next chapter, we will explore:

- Factory patterns with generics
- Repository pattern implementation
- Builder pattern with type-safe construction
- Generic utility functions
- Performance considerations with generics
- Common generic pitfalls and how to avoid them

Understanding generic patterns enables you to build reusable, type-safe abstractions that scale across large applications while maintaining compile-time type safety.

---